In [128]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
import os


os.environ["OPENAI_API_KEY"] = ""

question = """
    Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
    You may assume that each input would have exactly one solution, and you may not use the same element twice.

    You can return the answer in any order.

    Input: nums = [2,7,11,15], target = 9
    Output: [0,1]
    Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].
"""

expectedCode = """class Solution:
        def twoSum(self, nums: List[int], target: int) -> List[int]:
            hashmap = {}
            for i in range(len(nums)):
                hashmap[nums[i]] = i
            for i in range(len(nums)):
                complement = target - nums[i]
                if complement in hashmap and hashmap[complement] != i:
                    return [i, hashmap[complement]]

        """
expectedApproach = "A simple implementation uses two iterations. In the first iteration, we add each element's value as a key and its index as a value to the hash table. Then, in the second iteration, we check if each element's complement (target−nums[i]) exists in the hash table. If it does exist, we return current element's index and its complement's index. Beware that the complement must not be nums[i] itself!"

In [129]:
# Defining the interview state object

from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Literal
from enum import Enum


class Phase(str, Enum):
    INTRODUCTION = 'introduction'
    QUESTION  = 'question'
    APPROACH = 'approach'

class PhaseModel(BaseModel):
    phase : Literal['introduction', 'question', 'approach']
    
class Dialogue(BaseModel):
    dialogue: str = Field(description="The dialogue")
    speaker: Literal['interviewer', 'candidate'] = Field(description="The speaker of the dialogue")
    
class InterviewState(BaseModel):
    
    conversations: List[Dialogue] = Field(default_factory=list)
    question: str = question
    expectedApproach : str = expectedApproach
    expectedCode : str = expectedCode

    userCode : str = ""    
    phase : Phase = Phase.INTRODUCTION
    

class IntroductionResponse(BaseModel):
    dialogue : str
    

In [130]:
def readPrompt(filePath):
    content = open(filePath).read()
    return PromptTemplate.from_template(content)

def getModel():
    return  ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
    )


In [131]:
#Defining nodes of the workflow

def controlNode(state : InterviewState):
    prompt = readPrompt("prompts\control\classifyConversation.txt")
    model = getModel()
    chain = prompt | model.with_structured_output(PhaseModel)
    response = chain.invoke({'conversations':state.conversations})
    state.phase  =  Phase(response.phase)
    return state.dict()
    

def introductionNode(state : InterviewState):

    prompt = readPrompt("prompts\introduction\introduction.txt")
    model = getModel()

    chain = prompt | model

    response = chain.invoke({'conversations':state.conversations})
    state.conversations.append(Dialogue(dialogue=str(response.content), speaker="INTERVIEWER"))
    
    return state.dict()


def describeQuestionNode(state : InterviewState):
    prompt = readPrompt("prompts\questions\describeQuestion.txt")
    model = getModel()

    chain = prompt | model

    response = chain.invoke({'conversations':state.conversations, 'question':state.question})

    state.conversations.append(Dialogue(dialogue=str(response.content), speaker="INTERVIEWER"))
    
    return state.dict()


def describeApproachNode(state : InterviewState):
    prompt = readPrompt("prompts\questions\explainApproach.txt")
    model = getModel()

    chain = prompt | model

    response = chain.invoke({'conversations':state.conversations, 'question':state.question, 'expectedApproach':expectedApproach })

    state.conversations.append(Dialogue(dialogue=str(response.content), speaker="INTERVIEWER"))
    
    return state.dict()

In [132]:
from typing import Literal
from langgraph.graph import END

def checkPhase(state: InterviewState) -> Literal["introductionNode", "describeQuestionNode", "describeApproachNode", END]:
  
    if state.phase == Phase.INTRODUCTION :
        return "introductionNode"
    if state.phase == Phase.QUESTION:
        return "describeQuestionNode"
    if state.phase == Phase.APPROACH:
        return "describeApproachNode"
    
    return END

In [133]:
# Defining the graph

workflow = StateGraph(InterviewState)

workflow.add_node("controlNode", controlNode)
workflow.add_node("introductionNode", introductionNode)
workflow.add_node("describeQuestionNode", describeQuestionNode)
workflow.add_node("describeApproachNode", describeApproachNode)

workflow.add_edge(START, "controlNode")
workflow.add_conditional_edges(
    "controlNode",
    checkPhase,
)
graph = workflow.compile()

In [134]:
memory : dict[str, InterviewState]= {'1' : InterviewState()}

In [135]:
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs
import time

def read_text(text):
    client = ElevenLabs(
        api_key="",
    )
    response  = client.text_to_speech.convert(
        voice_id="cjVigY5qzO86Huf0OWal",
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        voice_settings=VoiceSettings(
            stability=0.1,
            similarity_boost=0.3,
            style=0.2,
        ),
    )
    save_file_path = f"output.mp3"

    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    os.system("start output.mp3") 


def process_user_input(user_input):
    try:
        state : InterviewState = memory['1']
        state.conversations.append(Dialogue(dialogue=user_input, speaker="candidate"))
        finalState = graph.invoke(state)
        memory['1'] = InterviewState(**finalState) 
        return memory['1']
    except Exception as e:
        print(e)

def chat():
    print(memory['1'])
    while True:
        user_input = input("")
        if user_input.lower() == 'exit':
            break
        response = process_user_input(user_input)
        text = (response.conversations[len(response.conversations)-1].dialogue)
        print(text)
        #read_text(text)
        

In [ ]:
chat()